# Введение или просто добавь воды

In [23]:
import requests
import numpy as np 
import pandas as pd

In [6]:
# this is a surprise tool that will help us later!
from fake_useragent import UserAgent

## 1.3 Наш первый запрос

Для наших благородных исследовательских целей нужно собрать данные по каждому мему с соответствующей ему страницы. Но для начала нужно получить адреса этих страниц. Поэтому открываем основную страницу со всеми выложенными мемами. Выглядит она следующим образом:

<img align="center" src="pictures/memes_main.png" height="500" width="500"> 

Отсюда мы и будем тащить ссылки на каждый из перечисленных мемов. Сохраним в переменную `page_link` адрес основной страницы и откроем её при помощи библиотеки `requests`.

In [69]:
page_link = 'http://knowyourmeme.com/memes/all/page/0'

In [70]:
response = requests.get(page_link)
response

<Response [403]>

А вот и первая проблема! Обращаемся к [главному источнику знаний](https://en.wikipedia.org/wiki/HTTP_403) и выясняем, что 403-я ошибка выдается сервером, если он доступен и способен обрабатывать запросы, но по некоторым личным причинам отказывается это делать. Попробуем выяснить, почему. Для этого посмотрим, как выглядел финальный запрос, отправленный нами на сервер, а в первую очередь - как выглядел наш User-Agent в глазах сервера.

In [71]:
for key, value in response.request.headers.items():
    print(key+": "+value)

User-Agent: python-requests/2.14.2
Accept-Encoding: gzip, deflate
Accept: */*
Connection: keep-alive


Похоже, мы недвусмысленно дали понять серверу, что мы сидим на питоне и используем библиотеку requests под версией 2.14.2. Скорее всего, это вызвало у сервера некоторые подозрения относительно наших благих намерений и он решил нас безжалостно отвергнуть. Для сравнения, можно посмотреть, как выглядят request-headers у здорового человека:

<img align="center" src="pictures/good_headers.png" height="800" width="800"> 

Очевидно, что нашему скромному запросу не тягаться с таким обилием мета-информации, которое передается при запросе из обычного браузера. К счастью, никто нам не мешает притвориться человечными и пустить пыль в глаза сервера при помощи генерации фейкового юзер-агента. Библиотек, которые справляются с такой задачей, существует очень и очень много, лично мне больше всего нравится [`fake-useragent`](https://pypi.python.org/pypi/fake-useragent). При вызове метода из различных кусочков будет генерироваться рандомное сочетание операционной системы, спецификаций и версии браузера, которые можно передавать в запрос:

In [72]:
UserAgent().chrome

'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'

In [73]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

Замечательно, наша небольшая маскировка сработала и обманутый сервер покорно выдал благословенный 200 ответ — соединение установлено и данные получены, всё чудесно! Посмотрим, что же все-таки мы получили.

In [74]:
html = response.content

In [75]:
html[:1000]

b'<!DOCTYPE html>\n<html xmlns:fb=\'http://www.facebook.com/2008/fbml\' xmlns=\'http://www.w3.org/1999/xhtml\'>\n<head>\n<meta content=\'text/html; charset=utf-8\' http-equiv=\'Content-Type\'>\n<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"c1a6d52f38","applicationID":"31165848","transactionName":"dFdfRUpeWglTQB8GDUNKWFRLHlcJWg==","queueTime":0,"applicationTime":66,"agent":""}</script>\n<script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(2),u=e(3),f=e("ee"

Выглядит неудобоваримо, как насчет сварить из этого дела что-то покрасивее? Например, красивый суп.

## 1.4. Красивый суп

<img align="center" src="https://www.crummy.com/software/BeautifulSoup/10.1.jpg" height="200" width="200"> 

Пакет **[bs4 , a.k.a BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/)** (тут есть гиперссылка на лучшего друга человека — документацию) был назван в честь стишка про красивый суп из Алисы в стране чудес.

Красивый суп. Эта совершенно волшебная библиотека, которая из сырого и необработанного HTML (или XML) кода страницы выдаст вам структурированный массив данных, по которому очень удобно искать необходимые теги, классы, атрибуты, тексты и прочие элементы веб страниц.

> Пакет под названием `BeautifulSoup` — скорее всего, не то, что вам нужно. Это третья версия (*Beautiful Soup 3*), а мы будем использовать четвертую. Так что нам нужен пакет `beautifulsoup4`. Чтобы было совсем весело, при импорте нужно указывать другое название пакета — `bs4`, а импортировать функцию под названием `BeautifulSoup`. В общем, сначала легко запутаться, но эти трудности нужно преодолеть.

In [76]:
from bs4 import BeautifulSoup

Передадим функции `BeautifulSoup` текст веб-страницы, которую мы недавно получили.

In [77]:
soup = BeautifulSoup(html,'html.parser') # В опции также можно указать lxml, 
                                         # если предварительно установить одноименный пакет

Получим что-то вот такое:
    
```
<!DOCTYPE html>

<html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"c1a6d52f38","applicationID":"31165848","transactionName":"dFdfRUpeWglTQB8GDUNKWFRLHkUNWUU=","queueTime":0,"applicationTime":24,"agent":""}</script>
<script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(2),u=e(3),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=
```

Стало намного лучше, не правда ли? Что же лежит в переменной `soup`? Невнимательный пользователь, скорее всего, скажет,что ничего вообще не изменилось. Тем не менее, это не так. Теперь мы можем свободно бродить по HTML-дереву страницы, искать детей, родителей и вытаскивать их! 

Например, можно бродить по вершинам, указывая путь из тегов.

In [78]:
soup.html.head.title

<title>All Entries | Know Your Meme</title>

Можно вытащить из того места, куда мы забрели, текст с помощью метода `text`.

In [79]:
soup.html.head.title.text

'All Entries | Know Your Meme'

Более того, зная адрес элемента, мы сразу можем найти его. Например, можно сделать это по классу. Следующая команда должна найти элемент, который лежит внутри тега `a` и имеет класс `photo`.

In [86]:
obj = soup.find('a', attrs = {'class':'photo'})
obj

<a class="photo left" href="/memes/events/al-franken-sexual-misconduct-allegations" target="_self"><img alt="Al Franken Accused Of Sexual Misconduct" data-src="http://i0.kym-cdn.com/featured_items/icons/wide/000/007/402/Screen_Shot_2017-11-16_at_2.00.18_PM.png" height="112" src="http://a.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Al Franken Accused Of Sexual Misconduct" width="198"/> <div class="info abs"> <div class="c"> Al Franken Accused Of Sexual Misconduct </div> </div> </a>

Однако, вопреки нашим ожиданиям, вытащенный объект имеет класс `"photo left"`. Оказывается, `BeautifulSoup4` расценивает аттрибуты `class` как набор отдельных значений, поэтому `"photo left"` для библиотеки равносильно `["photo", "left"]`, а указанное нами значение этого класса `"photo"` входит в этот список. Чтобы избежать такой неприятной ситуации и проходов по ненужным нам ссылкам, придется воспользоваться собственной функцией и задать строгое соответствие:

In [91]:
obj = soup.find(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
obj

<a class="photo" href="/memes/eas-new-handheld-console"><img alt="EA's new handheld console" data-src="http://i0.kym-cdn.com/entries/icons/medium/000/024/689/Capture.JPG" src="http://a.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="EA's new handheld console"/> <div class="entry-labels"> <span class="label label-deadpool"> Deadpool </span> </div> </a>

Полученный после поиска объект также обладает структурой bs4. Поэтому можно продолжить искать нужные нам объекты уже в нём! Вытащим ссылку на этот мем.

In [98]:
obj.attrs['href']

'/memes/eas-new-handheld-console'

Обратите внимание, что после всех этих безумных преобразований у данных поменялся тип. Теперь они `str`. Это означет, что с ними можно работать как с текстом и пускать в ход для отсеивания лишней информации регулярные выражения.  

In [100]:
print("Тип данных до вытаскивания ссылки:", type(obj))
print("Тип данных после вытаскивания ссылки:", type(obj.attrs['href']))

Тип данных до вытаскивания ссылки: <class 'bs4.element.Tag'>
Тип данных после вытаскивания ссылки: <class 'str'>


Если несколько элементов на странице обладают указанным адресом, то метод `find` вернёт только самый первый.  Чтобы найти все элементы с таким адресом, нужно использовать метод `findAll`. На выход будет выдан список!

In [103]:
soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])[:3]

[<a class="photo" href="/memes/eas-new-handheld-console"><img alt="EA's new handheld console" data-src="http://i0.kym-cdn.com/entries/icons/medium/000/024/689/Capture.JPG" src="http://a.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="EA's new handheld console"/> <div class="entry-labels"> <span class="label label-deadpool"> Deadpool </span> </div> </a>,
 <a class="photo" href="/memes/7-days-7-photos"><img alt="7 Days, 7 Photos" data-src="http://i0.kym-cdn.com/entries/icons/medium/000/024/688/7days.jpg" src="http://a.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="7 Days, 7 Photos"/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> </div> </a>,
 <a class="photo" href="/memes/pope-francis-signs-a-lamborghini"><img alt="Pope Francis Signs a Lamborghini" data-src="http://i0.kym-cdn.com/entries/icons/medium/000/024/687/pope.jpg" src="http://a.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="P